In [ ]:
import pandas as pd

In [ ]:
combustiveis_df = pd.read_excel("ca-2021-02.xlsx")
display(combustiveis_df.head())

In [ ]:
combustiveis_df['Ativo'] = True

display(combustiveis_df.head())

In [ ]:
# Criar uma coluna "Obs" que tenha nela escrito "MELHOR CIDADE" quando a coluna Municipio for igual a SAO PAULO
combustiveis_df['Obs'] = ["MELHOR CIDADE" if municipio == 'SAO PAULO' else None for municipio in combustiveis_df['Municipio']]
display(combustiveis_df.loc[combustiveis_df['Municipio'].isin(['SAO PAULO','INDAIATUBA', 'CAMPINAS', 'SALTO']), ['Municipio', 'Obs']])


In [ ]:
# se o valor de venda for maior que 6,5 reais, ele fala que ta caro, caso contrário, está barato
import numpy as np

combustiveis_df['Status do Valor de Venda'] = np.where(combustiveis_df['Valor de Venda'] > 6.5, 'Caro', 'Barato')
display(combustiveis_df[['Revenda', 'Valor de Venda', 'Status do Valor de Venda']])

In [ ]:
# Calcular postos de gasolina por habitante temos na amostragem de combustiveis 11/21

num_habitantes_df = pd.read_csv("ibge_num_habitantes_estimado.csv", sep=";")
num_habitantes_df.rename(columns={"Estado":"Estado - Sigla"}, inplace=True)
display(num_habitantes_df)

In [ ]:
colunas = ['Municipio', 'Estado - Sigla']
merge_df = combustiveis_df.merge(num_habitantes_df, how="inner", on=colunas)
display(merge_df)
print(merge_df.info())

In [ ]:
merge_df.dropna(axis='columns', inplace=True)
print(merge_df.info())

In [ ]:
colunas=['Regiao - Sigla', 'Nome da Rua', 'Numero Rua', 
         'Bairro', 'Cep', 'Produto', 'Data da Coleta', 'Valor de Venda',
         'Unidade de Medida', 'Bandeira', 'Ativo', 'Status do Valor de Venda']
merge_df.drop(labels=colunas, axis=1, inplace=True)
print(merge_df.info())

In [ ]:
display(merge_df.head(100))

In [ ]:
# Remover a linhas duplicadas
merge_df.drop_duplicates(inplace=True)
display(merge_df.head(100))

In [ ]:
postos_por_municipio_df = merge_df.groupby(by=['Estado - Sigla', 'Municipio', 'NumHabitantes2021']).count()
postos_por_municipio_df.reset_index(inplace=True)

postos_por_municipio_df.drop('CNPJ da Revenda', axis=1, inplace=True)
postos_por_municipio_df.rename(columns={"Revenda": "NumPostos"}, inplace=True)

postos_por_municipio_df['NumHabitantesPorPosto'] = postos_por_municipio_df['NumHabitantes2021'] / postos_por_municipio_df['NumPostos']

display(postos_por_municipio_df)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(combustiveis_df['Valor de venda'])

plt.title('Valor médio dos combustíveis - Nov/2021')
plt.xlabel('Preço (em reais)')
plt.ylabel('Quantidade de coletas')

plt.axvline(combustiveis_df['Valor de Venda'].mean(), color='red', linestyle='dashed', linewidth=5)

plt.show()


In [ ]:
c_mean = combustiveis_df['Valor de Venda'].groupby(by=combustiveis_df['Produto']).mean()
display(c_mean)
     
plt.figure(figsize=(7,5))

c_mean.plot(
    kind="barh",
    xlabel="Tipo de Combustível",
    ylabel="Preço reais/litro",
    color="red"
)

plt.grid()

plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(7,5))

c_mean.plot(
    kind="barh",
    xlabel="Tipo de Combustível",
    ylabel="Preço reais/litro",
    title="Média de preços por combustível",
    color="red",
    alpha=0.3
)

plt.grid()
sns.despine()

plt.show()

In [ ]:
display(c_mean)

excel = "por_litro.xlsx"
c_mean.to_excel(excel, "Sumário")

In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Color, Alignment

#Vai abrir o Excel no openpyxl
wb = load_workbook(excel)

#Pegar a planilha certa usando o Sheet Name
ws = wb['Sumário']

#Vamos pintar o cabeçalho da tabela de "cinza"
cinzinha = PatternFill("solid", fgColor="CCCCCC")
coords = ['A1', 'B1']
for coord in coords:
  ws[coord].fill = cinzinha

#Onde o preço do combustível for maior ou igual a 6,5 reais (6.5) pinta a fonte de vermelho e deixa negrito
MAX_ROW = ws.max_row
num_linha = 2
while (num_linha <= MAX_ROW):
  coord = 'B'+str(num_linha)
  if ws[coord].value >= 6.5:
    ws[coord].font = Font(bold=True, color="FF0000")
  num_linha = num_linha + 1

wb.save(excel)